In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os
sys.path.append(os.path.join(os.getcwd(), '../'))

In [3]:
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt

import json
from numpy import random
from dataclasses import dataclass

from model.utils import report_results
from model.gradient_boosting import train_gradient_boosting
from model.utils import MetricsResults

from sklearn.ensemble import HistGradientBoostingClassifier

from collections import defaultdict
from functools import reduce

DEFAULT_RANDOM_SEED = 774
random.mtrand._rand.seed(DEFAULT_RANDOM_SEED)
seed_list = random.random_integers(low=0, high=2**32 - 1, size=25)
warnings.filterwarnings("ignore")

category="min_tpm_5"

/var/folders/vp/tqkr07mj3cn274npl2jmvy5w0000gq/T/ipykernel_6801/2302145829.py:21: DeprecationWarning: This function is deprecated. Please call randint(0, 4294967295 + 1) instead
  seed_list = random.random_integers(low=0, high=2**32 - 1, size=25)


In [4]:
from model.gradient_boosting import TrainGradientBoostingResponse


def run_tests(genes: int, subtype: str, params: dict) -> TrainGradientBoostingResponse:
  data = pd.read_csv(f"../../preprocessed/{category}/genes.csv").drop(columns=["sample_id"])
  importances = json.loads(open(f"../../preprocessed/{category}/important_genes_logistic_f1.json").readline())

  chosen_genes = list(set([y["gene"] for x in [sex_values[:genes] for subtype_items in importances.values() for sex_values in subtype_items.values()] for y in x]))
  print(f"Total chosen genes: {len(chosen_genes)}")

  df = data[["subtype", "sex", *chosen_genes]]
  # df = pd.concat([df, df[df["subtype"] == subtype]])
  df["subtype_target"] = df["subtype"] == subtype
  return train_gradient_boosting("subtype_target", df, **params)

In [5]:
result = run_tests(genes=20, subtype="iAMP21", params={'class_weight': 'balanced', 'l2_regularization': 0.5, 'max_depth': 16, 'max_features': 0.1, 'learning_rate': 0.05})
result.acc

Total chosen genes: 336


0.6891304347826087

In [6]:
print(result.test_report)

              precision    recall  f1-score   support

       False       0.98      0.98      0.98       138
        True       0.40      0.40      0.40         5

    accuracy                           0.96       143
   macro avg       0.69      0.69      0.69       143
weighted avg       0.96      0.96      0.96       143

